In [2]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [6]:

options = Options()

options.add_argument('--user-data-dir=/Users/Administrator/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 1')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--start-maximized")

options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True

In [7]:
with open("CategoryUrls","rb") as f:
    category_urls = pickle.load(f)

In [8]:
all_details = {}
index = 0
count = 0
driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
for category_url in category_urls:
    print(f'category: {category_url.split("/")[-2]} --{count}/{len(category_urls)}')
    driver.get(category_url)
    time.sleep(3)
    try:
        sale_btn = driver.find_element(By.XPATH,"//*[@id='salepostcategory']/div/a/button")
        sale_btn.click()
    except:
        pass
    
    time.sleep(3)
    html_text = driver.page_source
    soup = BeautifulSoup(html_text,"lxml")
    products_part = soup.find("div",class_="news-detail-right-main")
    products = products_part.find_all("div",class_="col-lg-3 col-md-3 col-sm-12 col-xs-12")
    for product in products:
        details = {}

        details["name"] = product.find("h6").text
        details["link"] = product.find("a")["href"]
        details["price"] = product.find_all("div")[-1].text.strip()
        details["category"] = category_url.split("/")[-2]
        details["type"] = "sale"

        all_details[index] = details
        index += 1


    driver.get(category_url)
    time.sleep(3)
    try:
        buy_btn = driver.find_element(By.XPATH,"//*[@id='buypostcategory']/div/a/button")
        buy_btn.click()
    except:
        count+=1
        continue                             
    time.sleep(3)
    html_text = driver.page_source
    soup = BeautifulSoup(html_text,"lxml")
    products_part = soup.find("div",class_="news-detail-right-main")
    products = products_part.find_all("div",class_="col-lg-3 col-md-3 col-sm-12 col-xs-12")
   
    for product in products:
        details = {}
        
        details["name"] = product.find("h6").text
        details["link"] = product.find("a")["href"]
        details["price"] = product.find_all("div")[-1].text.strip()
        details["category"] = category_url.split("/")[-2]
        details["type"] = "buy"
        
        all_details[index] = details
        index += 1
    count+=1
    if count%5 == 0:
        driver.close()
        time.sleep(1)
        driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
    

category: pp --0/40
category: hdpe --1/40
category: ldpe --2/40
category: pvc --3/40
category: pet --4/40
category: lldpe --5/40
category: plastic-waste --6/40
category: ppcp --7/40
category: abs --8/40
category: pc --9/40
category: nylon --10/40
category: bopp --11/40
category: acrylic --12/40
category: upvc --13/40
category: polyester --14/40
category: hm --15/40
category: hips --16/40
category: pa --17/40
category: pom --18/40
category: gpps --19/40
category: machinery --20/40
category: mlp --21/40
category: xlpe --22/40
category: masterbatch --23/40
category: cpvc --24/40
category: rubber --25/40
category: ps --26/40
category: eva --27/40
category: biopolymers --28/40
category: filler --29/40
category: ppr --30/40
category: pbt --31/40
category: ptmt --32/40
category: uhmw --33/40
category: pvt --34/40
category: evoh --35/40
category: hmw --36/40
category: vae --37/40
category: aba --38/40
category: vhmw --39/40


In [12]:
df = pd.DataFrame().from_dict(all_details).T
df

,name,link,price,category,type
0,PP RAFFIA BAG CEMENT,https://www.plastic4trade.com/salepost-supplie...,₹ 20,pp,sale
1,COLOR SORTER,https://www.plastic4trade.com/salepost-supplie...,$ 12500,pp,sale
2,PP RAFIYA SCRAP,https://www.plastic4trade.com/salepost-supplie...,₹ 30,pp,sale
3,2000,https://www.plastic4trade.com/salepost-supplie...,$ 25,pp,sale
4,2000,https://www.plastic4trade.com/salepost-supplie...,₹ 25,pp,sale
...,...,...,...,...,...
2672,PBT,https://www.plastic4trade.com/buypost-buyer/pb...,₹ 100,pbt,sale
2673,"PIPES, FAUCETS, BIB COCKS",https://www.plastic4trade.com/buypost-buyer/pi...,₹ 100,ptmt,sale
2674,HD REPROCESS GRANULS,https://www.plastic4trade.com/buypost-buyer/pt...,₹ 180,ptmt,sale
2675,PVDF,https://www.plastic4trade.com/buypost-buyer/pv...,₹ 20,uhmw,sale


In [13]:
del df["type"]

In [14]:
df.to_csv("plastic4trade.csv",index=False)